In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2536)

In [15]:
x[:,:2]

matrix([[  964873.22929516,   972312.17461912],
        [  994524.53734304,   986088.26909181],
        [ 1788790.21047118,  1825481.53499941],
        [ 1913863.27568426,  1930608.06628325],
        [  479236.04022847,   483879.15207922],
        [  590670.59023864,   591010.11291544],
        [ 1191053.3033519 ,  1169782.09708927],
        [ 1137483.9931787 ,  1175772.74465763],
        [ 1070460.22107145,  1112237.026778  ],
        [ 1124795.19429022,  1119924.21221605],
        [ 1163233.58191659,  1186122.58699127],
        [ 1073910.67932845,  1069992.47063726],
        [  255130.71894429,   266958.43308127],
        [  293001.21750258,   286483.5685874 ],
        [  446125.70709491,   452312.53701773],
        [  398664.2214311 ,   371310.22587344],
        [  608958.28217456,   614685.86476824],
        [  619117.7872288 ,   609439.61846647],
        [  999946.46517858,   953982.8869574 ],
        [  834231.78932916,   871085.02742496],
        [ 1467815.19935094,  1494516.212

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [6e-02, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.09708985e+07  0.00000000e+00  5.74e+03 2.51e+03  1.00e+06     0s
   1  -2.43214904e+07 -1.17665792e+07  1.73e+03 7.57e+02  3.72e+05     0s
   2  -2.54711496e+07 -2.85621521e+07  7.10e+01 3.11e+01  8.58e+04     0s
   3  -2.63145940e+07 -2.69782536e+07  7.10e-05 3.11e-05  1.1

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[5.575285193091433e-08,
 839567.9035984569,
 21672.7415733995,
 48277.461332129715,
 105635.31297840297,
 0.00014873961598650676,
 0.00018520812179950719,
 1.306947436621362e-07,
 3.445206776641672e-09,
 735428.4371988671,
 3.4454590044561207e-09,
 3.4452741276357454e-09,
 3.4451231829402022e-09,
 3.46835454745161e-09,
 1422971.8301098465,
 4.512857657657628e-09,
 1.9120448892335704e-06,
 65336.77789664638,
 192075.14536456595,
 1382105.4297752813,
 0.00016007034355119965,
 372352.2405308863,
 882793.2096808681,
 126841.27009178478,
 6.0910579448427565e-05,
 7.237223074377935e-05,
 2.804280462506692e-05,
 560904.8409716677,
 214036.02669114264,
 4.518600481156054e-09,
 213760.72141857786,
 220649.20230918014,
 329650.1782481123,
 1.2609993134984855e-06,
 0.00016967753201764524,
 374564.5494698652,
 4.515412642010022e-09,
 348438.20973854273,
 3.0378814822081945e-06,
 339617.73836963886,
 687499.9960942519,
 136176.95218648852,
 7.90747228691165e-07,
 4.514587177154183e-09,
 569663.7848